## This module is used for testing the trained LSTM model on the test data

In [1]:
import os
import IPython
import IPython.display
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras
import tensorflow as tf

In [ ]:
batch_size=64
relative_rw = [0.2, 0.3, 0.4, 0.5, 0.6 ,0.7, 0.8]
test_accuracies = []
for i in range(len(relative_rw)):
    file_str = f"RWs_H_g_2_tadv_5min_rw_{relative_rw[i]}"
    file_str_test = "RWs_H_g_2_tadv_5min_rw_0.5"

    data_test = np.load(file_str_test+".npz")
    
    LSTM_save_name = os.getcwd()+ "/model_saves" + "/best_LSTM_"+file_str +"checkpoint.model.keras"

    for vars in data_test:
        print(vars)
    
    wave_data_train=data_test["wave_data_train"]
    wave_data_test=data_test["wave_data_test"]
    label_train=data_test["label_train"]
    label_test=data_test["label_test"]
    num_classes=2

    print(wave_data_train.shape)
    print(wave_data_test.shape)

    model_LSTM = keras.Sequential()
    model_LSTM.add(keras.layers.LSTM(10, input_shape = wave_data_train.shape[1:], return_sequences=True))
    model_LSTM.add(keras.layers.BatchNormalization())


    model_LSTM.add(keras.layers.LSTM(10, return_sequences=True)) #, return_sequences=True
    model_LSTM.add(keras.layers.BatchNormalization())


    model_LSTM.add(keras.layers.LSTM(10, return_sequences=True))#, return_sequences=True
    model_LSTM.add(keras.layers.BatchNormalization())


    model_LSTM.add(keras.layers.LSTM(10))
    model_LSTM.add(keras.layers.BatchNormalization())
    model_LSTM.add(keras.layers.Dropout(0.05))
    model_LSTM.add(keras.layers.Dense(num_classes, activation="sigmoid"))#


    model_LSTM.compile(loss="sparse_categorical_crossentropy",
                metrics=["sparse_categorical_accuracy"],
                optimizer="adam")

    print(model_LSTM.summary())

    model_LSTM = keras.models.load_model(LSTM_save_name)
    test_loss, test_acc = model_LSTM.evaluate(wave_data_test, label_test)
    preds_model = model_LSTM.predict(wave_data_test)

    print("Test accuracy", test_acc)
    print("Test loss", test_loss)

    test_accuracies.append(test_acc)
    np.savetxt(f"RWs_H_g_2_tadv_5min_rw_{relative_rw[i]}_scce_predictions.csv", np.asarray(preds_model), delimiter = ",")
    np.savetxt(f"RWs_H_g_2_tadv_5min_rw_{relative_rw[i]}_scce_labels.csv", np.asarray(label_test), delimiter = ",")
    
np.savetxt('Test_accuracies_RWs_H_g_2_tadv_5min_scce.csv', np.asarray(test_accuracies), delimiter = ",")

**The same file can be used for testing of the trained LSTM models for the different scenarios.**

In [ ]:
relative_rw = [0.2, 0.3, 0.4, 0.5, 0.6 ,0.7, 0.8]
for i in range(len(relative_rw)):
    file_str = f"RWs_H_g_2_tadv_5min_rw_{relative_rw[i]}_scce_"
    actual_labels = np.asarray(pd.read_csv(file_str + "labels.csv",header=None))
    predictions = np.asarray(pd.read_csv(file_str + "predictions.csv",header=None))
    predicted_labels = np.argmax(predictions, axis=1)

    confusion_matrix = metrics.confusion_matrix(actual_labels, predicted_labels)

    group_names = ['Correctly predicted','Incorrectly predicted',\
                   'Incorrectly predicted','Correctly predicted']
    group_counts = ["{0:0.0f}".format(value) for value in
                    confusion_matrix.flatten()]
    group_percentages = ["{0:.2%}".format(value) for value in
                         confusion_matrix.flatten()/np.sum(confusion_matrix)]
    labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in
              zip(group_names,group_counts,group_percentages)]
    labels = np.asarray(labels).reshape(2,2)
    yaxislabels = ['Rogue waves absent','Rogue waves present']
    xaxislabels = ['Predicted as absent','Predicted as present']
    plt.figure(figsize=[6,6])
    s = sns.heatmap(confusion_matrix, annot=labels, yticklabels=yaxislabels, xticklabels=xaxislabels, fmt='', cmap='Blues')
    s.set_xlabel("Predicted label", fontsize = 10)
    s.set_ylabel("True label", fontsize=10)
    plt.savefig(f"Confusion_matrix_RWs_H_g_2_tadv_5min_{relative_rw[i]}_.png", dpi=300)

**Sample confusion matrices for the forecasting process of the LSTM model using different proportions of rogue wave samples in the training process is provided here.**

<table>
  <tr>
    <td><img src = "lw_Confusion_matrix_RWs_H_g_2_tadv_5min_0.4_.png" width="400"> </td>
    <td><img src = "lw_Confusion_matrix_RWs_H_g_2_tadv_5min_0.5_.png" width="400"> </td>
    <td><img src = "lw_Confusion_matrix_RWs_H_g_2_tadv_5min_0.6_.png" width="400"> </td>
  </tr>
</table>

- **The correlation matrix plots here display the rogue forecasting efficiency for the LSTM model in a specific scenario where $t_{advance}$=5 minutes and $t_{training}$=20 minutes.**
- **The plots represent the forecasting results when different proportions of rogue wave windows are utilized in the training process.**
- **It can be observed that as the proportions of rogue wave examples in the training data increases($rw_{prop}$=0.6), better predictions can be attained for the rogue wave instances. However, the trained model in this case does not perform well for non rogue-wave instances. For lower proportions($rw_{prop}$=0.4), better predictions are observed for the non rogue-wave instances.**
- **The best overall accuracy is thus, observed for equal proportions of rogue wave and non rogue-wave instances in the training process (70%). However, if prediction of rogue waves is more crtical, we can opt for a model trained with larger number of rogue wave training examples.**